In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:


def createDataFrame(dir):
    image_path = []
    labels = []

    for label in os.listdir(dir):
        label_path = os.path.join(dir, label)
        if os.path.isdir(label_path):  
            for imagename in os.listdir(label_path):  
                image_path.append(os.path.join(label_path, imagename))
                labels.append(label)
    return image_path, labels




In [3]:

train_data_dir = 'D:/python/images/train'  
train = pd.DataFrame()
train['image'], train['label'] = createDataFrame(train_data_dir)
print(train)

                                          image     label
0            D:/python/images/train\angry\0.jpg     angry
1            D:/python/images/train\angry\1.jpg     angry
2           D:/python/images/train\angry\10.jpg     angry
3        D:/python/images/train\angry\10002.jpg     angry
4        D:/python/images/train\angry\10016.jpg     angry
...                                         ...       ...
28816  D:/python/images/train\surprise\9969.jpg  surprise
28817  D:/python/images/train\surprise\9985.jpg  surprise
28818  D:/python/images/train\surprise\9990.jpg  surprise
28819  D:/python/images/train\surprise\9992.jpg  surprise
28820  D:/python/images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [4]:
test_data_dir = 'D:/python/images/test'  
test = pd.DataFrame()
test['image'], test['label'] = createDataFrame(test_data_dir)
print(train)

                                          image     label
0            D:/python/images/train\angry\0.jpg     angry
1            D:/python/images/train\angry\1.jpg     angry
2           D:/python/images/train\angry\10.jpg     angry
3        D:/python/images/train\angry\10002.jpg     angry
4        D:/python/images/train\angry\10016.jpg     angry
...                                         ...       ...
28816  D:/python/images/train\surprise\9969.jpg  surprise
28817  D:/python/images/train\surprise\9985.jpg  surprise
28818  D:/python/images/train\surprise\9990.jpg  surprise
28819  D:/python/images/train\surprise\9992.jpg  surprise
28820  D:/python/images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [5]:
from tqdm import tqdm


In [6]:
def extract_features(images):
    features=[]
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale")
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [7]:
train_features = extract_features(train['image'])



00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 28821/28821 [05:40<00:00, 84.57it/s]

In [8]:
test_features = extract_features(test['image'])


00%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7066/7066 [01:23<00:00, 84.38it/s]

In [9]:
x_train=train_features/255.0
x_test=test_features/255.0

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le= LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [12]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [13]:
input_shape = (48, 48, 1)  # Example: 48x48 grayscale images

model = Sequential()

# Convolutional layers
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))

D:\python\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
# Ensure y_train and y_test are one-hot encoded
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Train the model
model.fit(x=x_train, y=y_train, batch_size=128, epochs=160, validation_data=(x_test, y_test))

model.save('trained_model_2.h5')


Epoch 1/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.2340 - loss: 1.8372 - val_accuracy: 0.2583 - val_loss: 1.8044
Epoch 2/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.2492 - loss: 1.8058 - val_accuracy: 0.2784 - val_loss: 1.7456
Epoch 3/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.2884 - loss: 1.7314 - val_accuracy: 0.3813 - val_loss: 1.5896
Epoch 4/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 292s 1s/step - accuracy: 0.3653 - loss: 1.6023 - val_accuracy: 0.4445 - val_loss: 1.4455
Epoch 5/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 296s 1s/step - accuracy: 0.4166 - loss: 1.5029 - val_accuracy: 0.4730 - val_loss: 1.3766
Epoch 6/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 298s 1s/step - accuracy: 0.4463 - loss: 1.4417 - val_accuracy: 0.4932 - val_loss: 1.3143
Epoch 7/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 300s 1s/step - accuracy: 0.4556 - loss: 1.4028 - val_accuracy: 0.5057 - val_loss: 1.2914
Epoch 8/160
226/226 ━━━━━━━━━━━━━━━━━━━━ 300s 1s/step - accuracy: 0.4825 - loss: 1.3632 - 